In [ ]:
#hide
import sys
if 'google.colab' in sys.modules:
    !pip install -Uqq fastai einops datasets axial_positional_embedding wandb
    !pip install -qq git+git://github.com/arampacha/reformer_fastai.git

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
#all_slow

# enwik8 - shared QK

In [ ]:
from fastai.text.all import *

from reformer_fastai.all import *

## Experiment Tracking

Make sure you have wandb and are logged in:

In [ ]:
# hide
# !wandb login

Load Experiment Tracking with Weights & Biases:

In [ ]:
import wandb

WANDB_NAME = 'enc_lm_enwik8_shared_qk_af'
GROUP = 'TEST'     # Group to add a run to, e.g. "LSH" for LSH experiments, "REVERSIBLE" for reversible layers
NOTES = 'TransformerLM on enwik8 sl 1024 w/ sharedQK using adafactor'
CONFIG = {}
TAGS = ['lm','shared_qk','enwik8']

Initialise wandb logging, pleaes **do not change** `project` or `entity` (that that everything gets logged to the same place)

In [ ]:
# wandb.init(reinit=True, project="reformer-fastai", entity="fastai_community", 
#            name=WANDB_NAME, group=GROUP, notes=NOTES, config=CONFIG, tags=TAGS)

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: fastai_community (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.14 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


## Download and Unpack enwik8 Data

Download and unzip enwik8 data

In [ ]:
path = untar_data('http://mattmahoney.net/dc/enwik8.zip', dest='/data')

## Prepare Data

In [ ]:
df = pd.DataFrame({'text':read_lines(path)})
df.head()

,text
0,"<mediawiki xmlns=""http://www.mediawiki.org/xml/export-0.3/"" xmlns:xsi=""http://www.w3.org/2001/XMLSchema-instance"" xsi:schemaLocation=""http://www.mediawiki.org/xml/export-0.3/ http://www.mediawiki.org/xml/export-0.3.xsd"" version=""0.3"" xml:lang=""en"">\n"
1,<siteinfo>\n
2,<sitename>Wikipedia</sitename>\n
3,<base>http://en.wikipedia.org/wiki/Main_Page</base>\n
4,<generator>MediaWiki 1.6alpha</generator>\n


In [ ]:
btt = ByteTextTokenizer(is_lm=True, add_bos=False, add_eos=False)

In [ ]:
%%time
df['toks'] = df['text'].apply(btt)
df['lens'] = df['toks'].apply(len)
df['lens_cum_sum'] = df.lens.cumsum()

CPU times: user 3min 4s, sys: 3.73 s, total: 3min 7s
Wall time: 3min 5s


In [ ]:
train_cutoff = df.lens.sum() - 10_000_000  # keep all but 10M characters for val and test
train_idxs = df.loc[df['lens_cum_sum'] < train_cutoff].index.values
train_idxs = list(range(0, max(train_idxs)))

remaining_idxs = len(df) - max(train_idxs)
validation_idxs = list(range(max(train_idxs), max(train_idxs) + int(remaining_idxs/2)))
test_idxs = list(range(max(validation_idxs), len(df)))

splits = [train_idxs, validation_idxs]

In [ ]:
tfms = [attrgetter("text"), btt]
dsets = Datasets(df, [tfms], splits=splits, dl_type=LMDataLoader)

In [ ]:
%%time
bs, sl = 2, 2**12
# pad_seq2seq = partial(pad_input, pad_idx=bte.pad_token_id, pad_fields=[0,1])
dl_kwargs = [{'lens':df['lens'].values[train_idxs]},
             {'val_lens':df['lens'].values[validation_idxs]}]
dls = dsets.dataloaders(bs=bs, seq_len=sl, dl_kwargs=dl_kwargs, shuffle_train=True, n_workers=2)

CPU times: user 39 s, sys: 1.25 s, total: 40.3 s
Wall time: 39.9 s


In [ ]:
#collapse_output
dls.show_batch(max_n=2,)

,text,text_
0,"\n <id>39547380</id>\n </page>\n* {{flagicon|Russia}} - [[Kostroma]] ([[Russia]]) since [[June 9]] [[2005]]\n </page>\n* [http://www.newadvent.org/fathers/0124.htm The Epistle of Barnabasn Catholic Encyclopedia]\nFrom three to six may play comfortably. It is possible to play with two, but not very interesting. Standard tournament games are played with four.\n\n <ip>69.196.4.226</ip>\n*Ph. Perrin (F), NASA/ESA, ISS assembly flight UF-2, STS-111/ISS, 05/06/02 - 19/06/02 (''Remark: was a CNES, not an ESA astronaut at the time of his mission'')\n* [[World Intellectual Property Organization]] (WIPO)\n* [http://www.nytimes.com/learning/general/onthisday/20050810.html ''The New York Times'': On This Day]\n==History==\n{{succession box | title=[[Lord Privy Seal]] | before=[[James Howard Harris, 3rd Earl of Malmesbury|The Earl of Malmesbury]] | after=[[Algernon Percy, 6th Duke of Northumberland|The Duke of Northumberland]] | years=1876&amp;ndash;1878}}\n===During the transition===\n|Hard\n==External links==\n </revision>\n[[bg:21 декември]]\n| March 4, 1861\n*[[1859]] - [[John Brown (abolitionist)|John Brown]], American abolitionist (hanged) (b. [[1800]])\n[[Category:1998 in law]]\n\n{{disambig}}</text>\n==International organization","<id>39547380</id>\n </page>\n* {{flagicon|Russia}} - [[Kostroma]] ([[Russia]]) since [[June 9]] [[2005]]\n </page>\n* [http://www.newadvent.org/fathers/0124.htm The Epistle of Barnabasn Catholic Encyclopedia]\nFrom three to six may play comfortably. It is possible to play with two, but not very interesting. Standard tournament games are played with four.\n\n <ip>69.196.4.226</ip>\n*Ph. Perrin (F), NASA/ESA, ISS assembly flight UF-2, STS-111/ISS, 05/06/02 - 19/06/02 (''Remark: was a CNES, not an ESA astronaut at the time of his mission'')\n* [[World Intellectual Property Organization]] (WIPO)\n* [http://www.nytimes.com/learning/general/onthisday/20050810.html ''The New York Times'': On This Day]\n==History==\n{{succession box | title=[[Lord Privy Seal]] | before=[[James Howard Harris, 3rd Earl of Malmesbury|The Earl of Malmesbury]] | after=[[Algernon Percy, 6th Duke of Northumberland|The Duke of Northumberland]] | years=1876&amp;ndash;1878}}\n===During the transition===\n|Hard\n==External links==\n </revision>\n[[bg:21 декември]]\n| March 4, 1861\n*[[1859]] - [[John Brown (abolitionist)|John Brown]], American abolitionist (hanged) (b. [[1800]])\n[[Category:1998 in law]]\n\n{{disambig}}</text>\n==International organization"
1,"of_encyclopedia_topics]] for an alternative method of finding subject areas to work on.\n <id>15903303</id>\nIn [[July 2004]], the U.S. Senate published an investigation into [[Riggs Bank]], a Washington-based bank into which most of Equatorial Guinea's oil revenues were paid until recently, and which also banked for Chile's Augusto Pinochet. The Senate report, as to Equatorial Guinea, showed that at least $35 million were siphoned off by Obiang, his family and senior officials of his regime. The president has denied any wrongdoing. While Riggs Bank in February 2005 paid $9 million as restitution for its banking for Chile's Augusto Pinochet, no restitution was made with regard to Equatorial Guinea, as reported in detail in [http://www.innercitypress.org/finwatch.html this Anti-Money Laundering Report from Inner City Press].\n\n <text xml:space=""preserve"">The '''Crookes radiometer''', also known as the '''light mill''' or '''solar engine''', consists of an airtight glass bulb, containing","f_encyclopedia_topics]] for an alternative method of finding subject areas to work on.\n <id>15903303</id>\nIn [[July 2004]], the U.S. Senate published an investigation into [[Riggs Bank]], a Washington-based bank into which most of Equatorial Guinea's oil revenues were paid until recently, and which also banked for Chile's Augusto Pinochet. The Senate report, as to Equatorial Guinea, showed that at least $35 million were siphoned off by Obiang, his family and senior officials of his

In [ ]:
vocab_sz = btt.vocab_size

In [ ]:
# xb, yb = dls.one_batch()
# xb.shape, yb.shape

In [ ]:
#hide
# del xb, yb
# torch.cuda.empty_cache()

## Training

In [ ]:
# wandb.init(reinit=True, project="reformer-fastai", entity="fastai_community", 
#            name=WANDB_NAME, group=GROUP, notes=NOTES, tags=TAGS, config=CONFIG)

In [ ]:
class NHashesConfig(ConfigBase):
    """
    Config for Synthetic Experiment.
    See https://arampacha.github.io/reformer_fastai/experiment.synthetic-task.html for details
    """
    _model = LSHLM
    _d = {
        'vocab_sz':256,
        'd_model':1024,
        'n_layers':3,
        'n_heads':8,
        'd_ff':4096,
        'attn_dropout':0.1,
        'ff_dropout':0.1,
        'emb_dropout':0.1,
        'tie_weights':True,
        'causal':True,
        'pos_enc':'axial',
        'max_seq_len':2**14,
        'axial_shape':None,
        'axial_emb_dims':None,
        'pad_idx':None,
        'prenorm':False,
        'attn_bias':False,
        'bucket_size':64,
        'use_lsh':True,
        'n_hashes':1,
        'seed':842,
    }
    @update_sig(_d)
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

In [ ]:
config = NHashesConfig()

In [ ]:
learn = Learner(dls, LSHLM.from_config(config),
                loss_func=CrossEntropyLossFlat(), opt_func=adafactor,
                metrics=[accuracy, perplexity, bpc]).to_fp16()

In [ ]:
# learn.lr_find()

In [ ]:
learn.fit(1, cbs=WandbCallback(log_model=False, log_preds=False))

RuntimeError: CUDA error: device-side assert triggered

In [ ]:
learn.recorder.plot_loss()